In [ ]:
import sys, time, random
import os
sys.path.append('../..')
import pybeamtools

In [ ]:
def time_fun():
    return time.time()*0.5

t = 0.0
def fixed_time():
    return t

In [ ]:
from pybeamtools.sim.core import SimulationEngine, SignalEngineOptions
sim = SimulationEngine(SignalEngineOptions(time_function=fixed_time))
sim.TRACE = True
from pybeamtools.sim.pddevices import *
ctx = SignalContext(se=sim)
echo = EchoDevice(ctx,options=EchoDeviceOptions(name='echo', data={'TEST:ECHO:1':5}))
echo2 = EchoDevice(ctx, options=EchoDeviceOptions(name='echo2', data={'TEST:ECHO:2':15}))
echo3= EchoDevice(ctx,options=EchoDeviceOptions(name='echo3', data={'TEST:ECHO:3':25}))
echo4 = EchoDevice(ctx, options=EchoDeviceOptions(name='echo4', data={'TEST:ECHO:4':35}))

var = 'TEST:GEN1'
data = {var:1.5}
def update_fun(t_sched, t_run, dep_data, self):
    return data

def read_fun(t_sched, t_run, channel_name, self):
    return data[var]

def write_fun(t_sched, t_run, value_dict, self):
    data[var] = value_dict[var]

def startup_fun(dev):
    import threading
    def updater():
        while True:
            data[var] = random.random()*10
            #print(f'Issuing update {data=}')
            for k in data:
                dev.ctx.issue_update(k)
            time.sleep(20)
    th = threading.Thread(target=updater, name='updater')
    th.daemon = True
    th.start()
gopt = GenericDeviceOptions(name='gen', update_fun=update_fun, read_fun=read_fun, write_fun=write_fun, startup_fun=startup_fun, channel_map={var:{}})
generic1 = GenericDevice(ctx, options=gopt)

proxy1 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy1', channel_map={'TEST:PROXY:1':{var: TRIGSPEC.PROPAGATE}}))
proxy2 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy2', channel_map={'TEST:PROXY:2':{'TEST:ECHO:2': TRIGSPEC.IGNORE}}))
proxy3 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy3', channel_map={'TEST:PROXY:3':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))
proxy4 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy4', channel_map={'TEST:PROXY:4':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))

In [ ]:
variables = ['X0', 'X1']
objectives = ['OBJ0']
test_variables = []

from pybeamtools.sim.templates import OptimizationTest
opt = OptimizationTest(variables, objectives)

In [ ]:
options_list = opt.setup()
options_list

In [ ]:
#gen_x0ai, gen_x0ao, gen_x1ai, gen_x1ao, gen_obj  = [GenericDevice(ctx, options=opt) for opt in options_list]
gen_x0ai, gen_x0ao, gen_x1ai, gen_x1ao, gen_obj = options_list

In [ ]:
for dev in options_list:
    dev.ctx = ctx
    ctx.add_device(dev)
    ctx.issue_update(dev.name)

In [ ]:
sim.latest_data

In [ ]:
sim.write_channel('X0:AI', 3.0)

In [ ]:
sim.latest_data

In [ ]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

In [ ]:
sim.write_channel('VARX0AI',random.random())

In [ ]:
sim.latest_data

In [ ]:
sim.latest_data